In [16]:
import pandas as pd
import requests
import requests_cache
import re
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pickle
import json

%matplotlib inline
requests_cache.install_cache("cache5")

In [3]:
recs = pd.read_json('clean_recs.json')

In [4]:
recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17775 entries, 0 to 9999
Data columns (total 11 columns):
calories       14090 non-null float64
categories     17775 non-null object
date           17775 non-null datetime64[ns]
desc           11918 non-null object
directions     17775 non-null object
fat            14047 non-null float64
ingredients    17775 non-null object
protein        14066 non-null float64
rating         17766 non-null float64
sodium         14090 non-null float64
title          17775 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 1.6+ MB


In [27]:
with open('even_more_recs.p', 'rb') as emr:
    dt = pickle.load(emr)

df2 = pd.DataFrame(list(dt.items()), columns=['title', 'link'])
data2 = df2.to_dict('records')

In [22]:
# TESTING

# url='https://www.epicurious.com/recipes/food/views/chicken-soup-with-caramelized-ginger'
# req=requests.get(url)
# soup = bs(req.text, 'lxml')


# nutri = soup.find('div', class_='nutrition content')
# nutri

# categories = [e.text for e in soup.find_all('dt', itemprop='recipeCategory')]
# date = soup.find('meta', itemprop='datePublished')['content']
# desc = soup.find('div', itemprop='description').p.text
# directions = [step.text.strip() for step in 
#               soup.find('div', class_='instructions').find_all('li', class_='preparation-step')]
# ingr = [i.text for i in
#        soup.find('div', class_='ingredients-info').find_all('li',class_='ingredient')]
# rating = float(soup.find('span', class_='rating').text.split('/')[0]) * 5 / 4


In [12]:
def get_info(rec_dict):
    
    url = rec_dict['link']
    
    req = requests.get(url)
    soup = bs(req.text, 'lxml')
    
    nutri = soup.find('div', class_='nutrition content')
    
    try: 
        calories = float(nutri.find('span', itemprop='calories').text)
    except AttributeError:
        return None
    try:
        fat = float(nutri.find('span', itemprop='fatContent').text.split()[0])
    except AttributeError:
        fat = None
    try:
        protein = float(nutri.find('span', itemprop='proteinContent').text.split()[0])
    except AttributeError:
        protein = None
    try:
        carb = float(nutri.find('span', itemprop='carbohydrateContent').text.split()[0])
    except AttributeError:
        carb = None
    try:
        sodium = float(nutri.find('span', itemprop='sodiumContent').text.split()[0])
    except AttributeError:
        sodium = None
    try:
        categories = [cat.text for cat in soup.find_all('dt', itemprop='recipeCategory')]
    except AttributeError:
        categories = None
    try:
        date = soup.find('meta', itemprop='datePublished')['content']
    except AttributeError:
        date = None
    try:
        desc = soup.find('div', itemprop='description').p.text
    except AttributeError:
        desc = None
    try:
        directions = [step.text.strip() for step in 
                  soup.find('div', class_='instructions').find_all('li', class_='preparation-step')]
    except AttributeError:
        directions = None
    try:
        ingredients = [i.text for i in
                    soup.find('div', class_='ingredients-info').find_all('li',class_='ingredient')]
    except AttributeError:
        ingredients = None
    try:
        rating = float(soup.find('span', class_='rating').text.split('/')[0]) * 5 / 4 #Scale to out of 5 rating
    except AttributeError:
        rating = None
 
        
    return (calories, fat, protein, carb, sodium, categories, date, desc, directions, ingredients, rating)
           
    

In [ ]:

# for rec in rec_to_scrape[5000:5500]:
#     res = get_info(rec)
#     print(rec_to_scrape.index(rec))
#     if res:
#         cal, fat, pro, so, cat, date, desc, di, ingr, rating = get_info(rec)

#         rec['calories'] = cal
#         rec['fat'] = fat
#         rec['protein'] = pro 
#         rec['sodium'] = so
#         rec['categories'] = cat
#         rec['date'] = date
#         rec['desc'] = desc
#         rec['directions'] = di
#         rec['ingredients'] = ingr
#         rec['rating'] = rating

In [17]:
# with open('new_recs_final.json', 'r') as file:
#     dat = json.load(file)

In [27]:
# df = pd.DataFrame(dat)
df.info()


# with open('new_recs_final_df.p', 'wb') as file:
#     pickle.dump(df, file, protocol=pickle.HIGHEST_PROTOCOL)

# with open('new_recs_final.json', 'w') as final:
#     json.dump(dat, final)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11173 entries, 0 to 11172
Data columns (total 13 columns):
calories       9500 non-null float64
carb           9486 non-null float64
categories     11173 non-null object
date           11173 non-null object
desc           9011 non-null object
directions     11173 non-null object
fat            9489 non-null float64
ingredients    11173 non-null object
link           11173 non-null object
protein        9492 non-null float64
rating         11154 non-null float64
sodium         9500 non-null float64
title          11173 non-null object
dtypes: float64(6), object(7)
memory usage: 1.1+ MB


In [20]:
for d in dat:
    req = requests.get(d['link'])
    soup = bs(req.text, 'lxml')
    try:
        d['carb'] = float(soup.find('span', itemprop='carbohydrateContent').text.split()[0])
    except AttributeError:
        d['carb'] = None
    print(d['title'],dat.index(d),d['carb'], req.from_cache)

Spring Pea Fish Chowder 0 42.0 True
Big-Batch Hard-Boiled Eggs 1 0.0 True
Date Night Pork Chop 2 31.0 True
Savory Dutch Baby for Two 3 22.0 True
Citrus Shrimp Rice Bowls 4 23.0 True
Cucumbers with Ajo Blanco Sauce 5 8.0 True
Make-Ahead Chicken Freezer Packets 6 10.0 True
Coconut-Strawberry Ice Cream Pie 7 45.0 True
Simplest Kale Salad 8 4.0 True
Mini Strawberry Eton Mess 9 34.0 True
Brown Butter Steel-Cut Oatmeal 10 20.0 True
One-Pot Turkey Bolognese with “Spaghetti” 11 None True
Summertime Sōmen Noodles 12 None True
Swordfish Steaks with Asparagus Salad 13 12.0 True
Slow-Roasted Bell Peppers 14 17.0 True
Pan-Seared Ribeye with Miso Butter 15 8.0 True
Honey-Sriracha Chicken Skewers 16 4.0 True
One-Pot Spring Pasta 17 81.0 True
Fruit Galette with Buckwheat Crust 18 36.0 True
Slow-Roasted Salmon with Harissa 19 None True
Crispy-Skinned Fish with Herb Sauce 20 10.0 True
Walnut-Crusted Salmon 21 17.0 True
Spicy Miso-Tahini Ramen 22 109.0 True
Lemony Asparagus 23 9.0 True
Baking-Sheet Bacon

Grilled Harissa Shrimp 179 None True
Pickled Nectarine Salad with Burrata 180 None True
Chocolate-Malt Ice Cream 181 20.0 True
Gingery Watermelon Paletas 182 28.0 True
Zingy Red Sauce 183 11.0 True
Grilled Beets with Burrata and Poppy Seed Vinaigrette 184 12.0 True
Cantaloupe in Pink Peppercorn Syrup 185 17.0 True
Simone's Shallot Vinaigrette 186 6.0 True
Albacore Tuna Sliders 187 None True
Cajun and Blackening Seasoning 188 25.0 True
Blackberry Farm Griddle Cakes 189 52.0 True
Quick-Pickled Shrimp 190 None True
Olive-Orange Vinaigrette 191 None True
Gingery Rhubarb Compote 192 57.0 True
Cilantro-Yogurt Sauce 193 4.0 True
Pickled Red Onions 194 2.0 True
Boxty 195 15.0 True
Dry-Roasted Brown Butter Onions 196 None True
Buttered Spinach with Vinegar 197 12.0 True
Gochujang-Date Sauce 198 20.0 True
Sesame-Pepper Bean Sprouts 199 None True
Wakame 200 0.0 True
Mulled White Wine with Pear Brandy 201 26.0 True
Pear Cranberry Sparklers 202 None True
Clementine Mulled Cider 203 None True
Ambros

Orange Shallot Mayonnaise 392 16.0 True
Whiskey Punch 393 15.0 True
Sesame Cumin Pita Crisps 394 40.0 True
Insalata Mista 395 7.0 True
Avocado Salsa 396 13.0 True
Armagnac Toffee Sauce 397 38.0 True
Gingered Cherry Sauce 398 None True
Roasted Pepper and Maple Barbecue Sauce 399 55.0 True
To Quick-Soak Dried Black-Eyed Peas 400 None True
Rosemary Toasts 401 81.0 True
Shallot Butter 402 3.0 True
Sole Goujons with Paprika Salt 403 None True
Lemon Cream 404 5.0 True
Cumin-Scented Rice Timbales 405 43.0 True
Fig, Ham and Nectarine Salad in Wine Syrup 406 17.0 True
Chocolate Maple Syrup 407 54.0 True
Rhubarb Raspberry Sauce 408 43.0 True
Coriander, Mint, and Chili Chutney 409 4.0 True
Clam and Oyster Chowder 410 10.0 True
Mustard Butter with Basil and Capers 411 4.0 True
To Toast and Skin Hazelnuts 412 None True
Open-Faced Pickled Mackerel and Apple Sandwiches 413 43.0 True
Herbed French Dressing 414 0.0 True
Poached Egg 415 0.0 True
Baguette Toasts 416 6.0 True
S'mores with a Twist 417 None

Chicken & Squash Cacciatore, Mushrooms, Tomatoes, Olives, Bread 569 48.0 True
Radicchio Salad with Pickled Grapes and Goat Cheese 570 21.0 True
Kale, Apple, Walnut and Sumac Onion Tabbouleh 571 5.0 True
Classic Dill Pickles 572 None True
Mixed Beans with Peanuts, Ginger, and Lime 573 12.0 True
Solterito De Quinua (Quinoa Solterito) 574 53.0 True
Butterflied Trout with Spicy Lettuce, Celery, and Herbs 575 8.0 True
Herby Black Rice Salad with Radishes and Ricotta Salata 576 41.0 True
Black Bass With Scallion-Chile Relish 577 3.0 True
Grilled Oregano Chicken 578 3.0 True
Collard Greens and Kale Pesto 579 9.0 True
Charred Romaine Greek Salad With Quinoa-Crusted Feta 580 80.0 True
Beets Don't Kale My Vibe Salad 581 None True
Green Salad with Prosciutto Vinaigrette 582 47.0 True
Baked Snapper with Harissa, New Potatoes, and Spring Onions 583 22.0 True
Shrimp And Fava Beans 584 29.0 True
Roasted Carrots with Fennel-Brown Butter 585 17.0 True
Coconut Tapioca With Pineapple, Mango, And Lime 586

Raspberry & Banana No-Knead Breakfast Loaf  744 32.0 True
Golden Milk 745 17.0 True
Green Pea, Asparagus, and Parsley Soup   746 None True
Blood Orange–Chia Pudding 747 54.0 True
Persian-Spiced Chicken with Spaghetti Squash 748 28.0 True
Spaghetti Squash “Noodle” Bowls with Skirt Steak 749 26.0 True
Chickpea and Eggplant Salad 750 241.0 True
Fig-Braised Chicken with Spiced Walnuts 751 16.0 True
Farro with Pistachios, Mixed Herbs, and Golden Raisins 752 39.0 True
Crispy Chicken Stew with Lemon, Artichokes, Capers, and Olives 753 20.0 True
Wild Mushroom Noodle Soup 754 21.0 True
Cauliflower With Quinoa, Prunes and Peanuts 755 24.0 True
Broccoli-Quinoa Salad with Buttermilk Dressing 756 51.0 True
Stellar Quinoa Burger 757 58.0 True
Miso-Turmeric Dressing 758 9.0 True
Tropical Carrot, Turmeric, and Ginger Smoothie 759 25.0 True
Avocado, Kale, Pineapple, and Coconut Smoothie 760 13.0 True
Toasted Nori Mayonnaise 761 0.0 True
Banana, Coffee, Cashew, and Cocoa Smoothie 762 22.0 True
Salmon Te

...

Salmon with Endive, Dill, and Cream 10461 7.0 True
Baked Shrimp with Scallions, Tomatoes, and Feta 10462 11.0 True
Shrimp Cakes with Chili-Lime Cream Sauce 10463 16.0 True
Nuked Nachos 10464 90.0 True
Drunken Sausages and Peppers with Hero Rolls 10465 2.0 True
Mini Fudgey Chocolate Cakes 10466 53.0 True
Frozen Chocolate-Peanut Butter Decadence 10467 26.0 True
Sugar-Crusted Chocolate Chip and Cherry Bread Puddings 10468 38.0 True
Sauteed Apples 10469 40.0 True
Pan-Roasted Swordfish Steaks with Mixed-Peppercorn Butter 10470 1.0 True
Sage and Pancetta Biscuits with Fontina Cheese 10471 20.0 True
Poached Oysters in Fennel-Saffron Soup 10472 12.0 True
Ciabatta Pizza with Gorgonzola, Walnut Pesto, and Pears 10473 37.0 True
All-Season Bean Salad 10474 23.0 True
Lemon Rice 10475 43.0 True
Orange Streamer Salad 10476 31.0 True
Chris Gargone's Chocolate Chews 10477 12.0 True
Scarborough Fair Shortbread 10478 12.0 True
Wildflower-Honey Semifreddo with Honey Sesame Wafers 10479 63.0 True
Chocolate

Yeast Rolls 10617 19.0 True
Ginger Doughnuts 10618 18.0 True
Molasses Crinkles 10619 8.0 True
Anise Sesame Cookies 10620 10.0 True
Lemon-Glazed Persimmon Bars 10621 20.0 True
Almond Bar Cookies 10622 9.0 True
Creamed Spinach 10623 10.0 True
Roasted Deviled Potatoes 10624 20.0 True
Potato Parsnip Mash 10625 47.0 True
Spicy Tomato Soup 10626 29.0 True
Black Bean Soup 10627 26.0 True
Cheddar Cheese and Onion Pie 10628 43.0 True
Spicy Toasted Pecans 10629 5.0 True
Mushrooms with Garlic Butter 10630 14.0 True
Spanish Olive and Cream Cheese Canapes 10631 4.0 True
Mincemeat Ice Cream 10632 33.0 True
Old-Fashioned Gingerbread with Molasses Whipped Cream 10633 64.0 True
Chocolate-Raspberry Cakes 10634 38.0 True
Dark Chocolate Mousse with Sugared Kumquats 10635 38.0 True
Persimmon Salsa 10636 11.0 True
Yukon Gold Potato and Wild Mushroom Gratin 10637 37.0 True
Trio of Winter Greens 10638 3.0 True
Perfect Rice 10639 31.0 True
Creamy Mashed Potatoes with Goat Cheese and Fresh Sage 10640 43.0 True


Asian Noodle, Mushroom, and Cabbage Salad 10784 50.0 True
Peking-Style Chicken Wraps 10785 69.0 True
Mussels with Sausage, Coconut Milk and Lime 10786 12.0 True
Salmon with Soy-Honey and Wasabi Sauces 10787 21.0 True
Scallops and Leeks in Star Anise-Orange Sauce 10788 30.0 True
Crispy Noodle Pancake 10789 63.0 True
Fried Shrimp with Peanut Sauce 10790 19.0 True
Ginger Fried Rice with Shiitake Mushrooms 10791 30.0 True
Edamame 10792 6.0 True
Rice Noodles with Garlic and Herbs 10793 60.0 True
Beef Sates with Southeast Asian Sauce 10794 1.0 True
Asian Risotto with Pancetta and Mussels 10795 100.0 True
Spicy Pork and Kumquat Stir-Fry 10796 17.0 True
Shrimp Salad 10797 10.0 True
Vanilla Thins 10798 3.0 True
Asian Five-Spice Chocolate Cake 10799 30.0 True
Tofu and Leek Stir-Fry with Ground Beef 10800 15.0 True
Paniolo Rack of Lamb 10801 34.0 True
Seared Tuna Pepper Steaks 10802 2.0 True
Feta-Pine Nut Spread 10803 5.0 True
Papaya Spring Rolls with Peanut Sauce 10804 11.0 True
Cola-Braised Por

Minted Eggplant Rounds 10952 15.0 True
Minted Lemongrass Sorbet 10953 43.0 True
Ribs with Orange-Ginger Glaze 10954 60.0 True
Peanut Dipping Sauce 10955 14.0 True
Sesame Spinach and Broccoli 10956 17.0 True
Szechuan Shrimp with Peppers 10957 9.0 True
Crab Pot Stickers with Sesame-Ginger Dipping Sauce 10958 123.0 True
Lemongrass Tofu with Mushrooms 10959 29.0 True
Szechwan Carrot Soup 10960 18.0 True
Goreng Bawang: Crisp-Fried Onions 10961 11.0 True
Turmeric-Kale Fried Rice 10962 82.0 True
Filipino-Style Meatloaf (Embutido) 10963 17.0 True
Super Green Stir-Fry 10964 73.0 True
Wonton Soup with Mushroom-Zucchini "Meatballs" 10965 66.0 True
Coconut Soba Noodles with Ginger Broccoli 10966 55.0 True
Hot-Honey Shrimp with Bok Choy 10967 42.0 True
Brussels Sprouts Salad with Szechuan Peppercorn and Celery 10968 9.0 True
Crunchy Turkey Salad with Peanut Dressing 10969 5.0 True
Short-Grain Rice 10970 52.0 True
Garlicky Spinach 10971 3.0 True
Sesame Carrots 10972 3.0 True
Steak Skewers with Scall

Angled Loofah Bread-and-Butter Pickles 11125 42.0 True
Chinese Barbecued Pork 11126 16.0 True
Scallion Cilantro Pancakes 11127 4.0 True
Sweet Chili Sauce 11128 52.0 True
Silky Peanut Butter Dressing 11129 17.0 True
Ugly Baby’s Red Curry Paste 11130 25.0 True
Filipino Spiced Vinegar 11131 5.0 True
Chicken Satay 11132 8.0 True
Smashed Cucumber Salad with Hot Vinegar 11133 7.0 True
Coconut Shrimp Pancakes with Fresh Herbs 11134 19.0 True
Asian-Style Poached Chicken 11135 10.0 True
Ensaymadas 11136 34.0 True
Roast Pork Belly with Chile Vinegar 11137 13.0 True
Cooked Semi-Polished Rice (Haiga Mai) 11138 64.0 True
Miso-Mustard Dressing 11139 6.0 True
Cornmeal Bao With Turkey and Black Pepper Sauce 11140 32.0 True
Alt-Grain Porridge With Kimchi and Jammy Eggs 11141 34.0 True
Stuffing Fried Rice 11142 124.0 True
Sesame-Lime Vinaigrette 11143 16.0 True
Spiced Custard Pie with Sesame Crust 11144 41.0 True
Crushed Cucumbers with Lime Pickle and Coconut 11145 5.0 True
Chopstick Ready Rice 11146 39

In [17]:
test = pd.DataFrame(rec_new)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11189 entries, 0 to 11188
Data columns (total 12 columns):
calories       9516 non-null float64
categories     11189 non-null object
date           11189 non-null object
desc           9027 non-null object
directions     11189 non-null object
fat            9505 non-null float64
ingredients    11189 non-null object
link           11189 non-null object
protein        9508 non-null float64
rating         11170 non-null float64
sodium         9516 non-null float64
title          11189 non-null object
dtypes: float64(5), object(7)
memory usage: 1.0+ MB


In [21]:
# with open('rec_info.p', 'rb') as file:
#     rec_new = pickle.load(file)

# rec_new.extend(temp)
rec_new  = [e for e in rec_new if len(e) != 2]
